In [1]:
import pickle
import time
import gc

In [2]:
max_size = 65
max_size = 39
print("max_size:", max_size)
num_try = 8
print("number of try:", num_try)

max_size: 39
number of try: 8


In [3]:
from libs_qrem import MooneyEtalFilter
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter

In [4]:
def get_info(mitigator):
    return {# "exact_one_norm_of_reduced_inv_A": mitigator.exact_one_norm_of_reduced_inv_A(),
            "mitigated_hist": mitigator.mitigated_hist(),
            "times": mitigator.times(),
            # "mitigation_stddev": mitigator.mitigation_stddev(norm_type = "exact"),
            }

In [5]:
def results_to_info(n):
    
    with open("pkls/results/results_" + str(n) + "-qubit.pkl", "rb") as f:
        results_list = pickle.load(f)
        f.close()
    
    mooney01_ghz_mqc_info_list = []
    for j in range(num_try):
        mooney01_ghz_mqc_info = {}
        mitigator = MooneyEtalFilter(n, TensoredMeasFitter(results_list[j]["mit"], mit_pattern=[[k] for k in range(n)]).cal_matrices)
        _ = mitigator.apply(results_list[j]["ghz"].get_counts(), threshold=0.1)
        mooney01_ghz_mqc_info["ghz"] = get_info(mitigator)
        mooney01_ghz_mqc_info["mqc"] = []
        for i, mqc_hist in enumerate( results_list[j]["mqc"].get_counts() ):
            _ = mitigator.apply(mqc_hist)
            mooney01_ghz_mqc_info["mqc"].append( get_info(mitigator) )
        mooney01_ghz_mqc_info_list.append(mooney01_ghz_mqc_info)
    with open("pkls/mooney01/info_list_" + str(n) + "-qubit.pkl", "wb") as f:
        pickle.dump(mooney01_ghz_mqc_info_list, f)
        f.close()

# for n in range(1, max_size + 1):
    t1 = time.perf_counter()
    results_to_info(n)
    t2 = time.perf_counter()
    print(n, "qubit finished ( time:", t2 - t1, "s )")